In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import zipfile
import random
import shutil

In [ ]:
path_cats_and_dogs = '/kaggle/input/dogs-vs-cats/train.zip'
shutil.rmtree('/tmp')

local_zip = path_cats_and_dogs
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/source')
zip_ref.close()

In [ ]:
os.mkdir('/tmp/train')
os.mkdir('/tmp/train/cats')
os.mkdir('/tmp/train/dogs')
os.mkdir('/tmp/validation')
os.mkdir('/tmp/validation/cats')
os.mkdir('/tmp/validation/dogs')
os.mkdir('/tmp/test')
os.mkdir('/tmp/test/cats')
os.mkdir('/tmp/test/dogs')

In [ ]:
import random

In [ ]:
#train contains all cat and dog pictures
#function to take from source directory and split by cat and dog
def split_dir(source,train_cat_dir,train_dog_dir,val_cat_dir,val_dog_dir,size=0.7):
    files = os.listdir(source) #list of files in source dir
    split_num = int(size*len(files)) # no. of files to go into train dir
    train_files = random.sample(files,split_num) # taking random number of files to go in training_dir
    val_files = list(set(files)-set(train_files)) # the rest go to validation dir
    
    for pic in train_files:
        if pic[:3] == 'cat':
            shutil.copy(os.path.join(source,pic),train_cat_dir)
        elif pic[:3] == 'dog':
            shutil.copy(os.path.join(source,pic),train_dog_dir)
    
    for pic in val_files:
        if pic[:3] == 'cat':
            shutil.copy(os.path.join(source,pic),val_cat_dir)
        elif pic[:3] == 'dog':
            shutil.copy(os.path.join(source,pic),val_dog_dir)

In [ ]:
os.listdir('/tmp/source/train')[0][:3]

In [ ]:
SOURCE = '/tmp/source/train'
train_cat = '/tmp/train/cats'
train_dog = '/tmp/train/dogs'
val_cat = '/tmp/validation/cats'
val_dog = '/tmp/validation/dogs'

In [ ]:
split_dir(SOURCE,train_cat,train_dog,val_cat,val_dog)

In [ ]:
print(len(os.listdir(SOURCE)))
print(len(os.listdir(train_cat))) 
print(len(os.listdir(train_dog)))
print(len(os.listdir(val_cat)))
print(len(os.listdir(val_dog)))
#expecting 17,500 total in training and 7,500 in validation

# Visualising the images

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
#showing 8 random cat images
plt.figure(figsize=(16,16))

#enumerate with first item being a num between 0 and 7 and second being 8 random numbers between 0 and the length of the train_cat directory
for i,cat in enumerate(np.random.randint(0,len(os.listdir(train_cat)),8)):
  fig = plt.subplot(4,4,i+1) #rows,column,index. Index starts at 1 in matplotlib
  fig.axis('off')
  img = mpimg.imread(os.path.join(train_cat, os.listdir(train_cat)[cat])) # reads image from file path
  fig.imshow(img) # show image

In [ ]:
#showing 8 random dog images
plt.figure(figsize=(16,16))
for i,dog in enumerate(np.random.randint(0,len(os.listdir(train_dog)),8)):
  fig = plt.subplot(4,4,i+1) #rows,column,index. Index starts at 1 in matplotlib
  fig.axis('off')
  img = mpimg.imread(os.path.join(train_dog, os.listdir(train_dog)[dog])) # reads image from file path
  fig.imshow(img) # show image

# Preprocessing images using generators

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir='/tmp/train'
validation_dir = '/tmp/validation'

In [ ]:
#will augment the training images by rotating,shifting,skewing and zooming the images
train_datagen = ImageDataGenerator(rescale=1/255,
                                  rotation_range=45,
                                  shear_range=0.2,
                                  fill_mode='nearest') 

val_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    target_size=(150,150),#making all the images 150*150 pixels
                                                    class_mode='binary')

val_generator = val_datagen.flow_from_directory(validation_dir,
                                                    batch_size=20,
                                                    target_size=(150,150),#making all the images 150*150 pixels
                                                    class_mode='binary')

# Building model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [ ]:
model = Sequential()

model.add(layers.Conv2D(16,(3,3),activation ='relu',input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(32,(3,3),activation ='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation ='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras import optimizers

In [ ]:
model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(learning_rate=0.001),
             metrics=['acc'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(mode='max',monitor='val_acc',patience=10,verbose=1)

In [ ]:
history = model.fit(train_generator,
                    validation_data=val_generator,
                    batch_size=512,
                    epochs=100,
                   callbacks=[early_stop])

# Predicting

In [ ]:
train_generator.class_indices

In [ ]:
path_test = '/kaggle/input/dogs-vs-cats/test1.zip'
local_zip = path_test
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/source')
zip_ref.close()

In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
# predicting images
pred = []
for pic in os.listdir('/tmp/source/test1'):
    path = '/tmp/source/test1/'+ pic
    img=image.load_img(path, target_size=(150, 150))

    x=image.img_to_array(img)
    x=np.expand_dims(x, axis=0)
    x=x/255

    pred.append(int(model.predict_classes(x)))

In [ ]:
id_ = [int(name[:-4]) for name in os.listdir('/tmp/source/test1')]
label = pred

In [ ]:
submission = pd.DataFrame({'id':id_,
             'label':pred})

In [ ]:
submission.sort_values('id',).to_csv('submission.csv',index=False)